In [589]:
import pandas as pd 
# pd.options.mode.chained_assignment = None  
import numpy as np 
from datetime import datetime

In [590]:
# Data Paths 
data = '../data/'

eth_historical = data+'eth_historical.pickle'
osqueeth_historical = data+'osqueeth_historical.pickle'

#constants 
funding_period = 420 #hours  

In [591]:
eth_historical = pd.read_pickle(eth_historical)
eth_historical = eth_historical.set_index('Date')
eth_historical.head() 

,Contract_Name,Contract_Address,Ticker_Symbol,Currency,Price
Date,,,,,
2022-03-06,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2625.5842
2022-03-05,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2651.4854
2022-03-04,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2841.9568
2022-03-03,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2935.3157
2022-03-02,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2974.9960


In [592]:
osqueeth_historical = pd.read_pickle(osqueeth_historical)
osqueeth_historical = osqueeth_historical.set_index('Date')
osqueeth_historical

,Contract_Name,Contract_Address,Ticker_Symbol,Currency,Price
Date,,,,,
2022-03-06,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,577.88650
2022-03-05,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,596.35730
2022-03-04,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,643.96970
2022-03-03,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,716.46857
2022-03-02,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,763.19464
2022-03-01,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,741.45660
2022-02-28,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,637.26117
2022-02-27,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,611.80510
2022-02-26,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,679.13830


In [593]:
historical = eth_historical.join(osqueeth_historical,how='inner',on=['Date'],lsuffix='_left', rsuffix='_right')
historical.head() 

,Contract_Name_left,Contract_Address_left,Ticker_Symbol_left,Currency_left,Price_left,Contract_Name_right,Contract_Address_right,Ticker_Symbol_right,Currency_right,Price_right
Date,,,,,,,,,,
2022-03-06,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2625.5842,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,577.88650
2022-03-05,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2651.4854,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,596.35730
2022-03-04,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2841.9568,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,643.96970
2022-03-03,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2935.3157,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,716.46857
2022-03-02,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2974.9960,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,763.19464


Index price: the Squeeth index price is ETH² <br>
Mark price: Mark is the current trading price of Squeeth <br>
implied funding = ln(currentMark / currentIndex) / funding period <br>
const funding_period = 420 hours 

In [594]:
historical_reduced = historical[['Ticker_Symbol_left','Price_left','Ticker_Symbol_right','Price_right']]
historical_reduced.columns = ['WETH','WETH_PRICE','MARK_TICKER','MARK_TICKER_PRICE']
historical_reduced['INDEX_TICKER'] = 'WETH^2'
historical_reduced['INDEX_TICKER_PRICE'] = [float(value)**2 for value in historical_reduced['WETH_PRICE'].to_list()]
mark_ticker_price = [value*10000 for value in historical_reduced['MARK_TICKER_PRICE'].to_list()]
index_ticker_price = [value for value in historical_reduced['INDEX_TICKER_PRICE'].to_list()]
historical_reduced['IMPLIED_FUNDING'] = np.log(np.array(mark_ticker_price),np.array(index_ticker_price))/funding_period
historical_reduced

,WETH,WETH_PRICE,MARK_TICKER,MARK_TICKER_PRICE,INDEX_TICKER,INDEX_TICKER_PRICE,IMPLIED_FUNDING
Date,,,,,,,
2022-03-06,WETH,2625.5842,oSQTH,577.88650,WETH^2,6.893692e+06,0.037071
2022-03-05,WETH,2651.4854,oSQTH,596.35730,WETH^2,7.030375e+06,0.037146
2022-03-04,WETH,2841.9568,oSQTH,643.96970,WETH^2,8.076718e+06,0.037329
2022-03-03,WETH,2935.3157,oSQTH,716.46857,WETH^2,8.616078e+06,0.037583
2022-03-02,WETH,2974.9960,oSQTH,763.19464,WETH^2,8.850601e+06,0.037733
2022-03-01,WETH,2892.4531,oSQTH,741.45660,WETH^2,8.366285e+06,0.037664
2022-02-28,WETH,2546.8340,oSQTH,637.26117,WETH^2,6.486363e+06,0.037304
2022-02-27,WETH,2707.0608,oSQTH,611.80510,WETH^2,7.328178e+06,0.037207
2022-02-26,WETH,2794.3835,oSQTH,679.13830,WETH^2,7.808579e+06,0.037455


In [595]:
#funding 
current_funding = pd.read_csv(data+'historical-funding.csv')
current_funding = current_funding.rename(columns={'DateTime': 'Date'})
current_funding = current_funding.set_index('Date')
current_funding.columns = ['Current_Funding','Daily_Funding']
current_funding.index = [date[:10] for date in current_funding.index]
#implied volatility 
current_implied_volatility = pd.read_csv(data+'historical-implied-volat.csv')
current_implied_volatility = current_implied_volatility.rename(columns={'DateTime':'Date'})
current_implied_volatility = current_implied_volatility.set_index('Date')
current_implied_volatility.columns = ['Current_IV','Daily_IV']
current_implied_volatility.index = [date[:10] for date in current_implied_volatility.index]

#squeeth_analytics 
squeeth_analytics = historical_reduced.join(current_funding,how='inner',on=['Date'],lsuffix='_left', rsuffix='_right')
squeeth_analytics = squeeth_analytics.join(current_implied_volatility,how='inner',on=['Date'],lsuffix='_left', rsuffix='_right')
squeeth_analytics = squeeth_analytics[['WETH_PRICE','INDEX_TICKER_PRICE','MARK_TICKER_PRICE','Current_Funding','Daily_Funding','Daily_IV']]
squeeth_analytics.head()

,WETH_PRICE,INDEX_TICKER_PRICE,MARK_TICKER_PRICE,Current_Funding,Daily_Funding,Daily_IV
Date,,,,,,
2022-03-06,2625.5842,6.893692e+06,577.8865,0.231,0.291,103.12
2022-03-06,2625.5842,6.893692e+06,577.8865,0.231,0.291,101.07
2022-03-06,2625.5842,6.893692e+06,577.8865,0.231,0.291,99.61
2022-03-06,2625.5842,6.893692e+06,577.8865,0.231,0.291,98.72
2022-03-06,2625.5842,6.893692e+06,577.8865,0.231,0.291,98.62
